In [36]:
import json
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
#import yfinance as yf
import bs4 as bs
import datetime

In [37]:
# Read and print the stock tickers that make up S&P500
response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(response.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [75]:
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
tickers = [s.replace('\n', '') for s in tickers]
tickers = sorted(tickers)
stocks = pd.DataFrame(tickers, columns=['Ticker'])
tickers.index('A') #= ', '.join(tickers)

0

In [81]:
# api request
symbol = 'AAPL'
token = '1xAmWTsYoKxpEBAgSErNPSb3rAdmeGxL'
api_url = f'https://financialmodelingprep.com/api/v3/quote/{symbol}?apikey={token}'

session = requests.session()
request = session.get(api_url, timeout=15)
data = request.json()
#data#[0]['price']
#print(data)
tickers.index(symbol)

2

In [84]:
# Get the prices and info for this ticker
marketCap= data[list(symbol.split(",")).index(symbol)]['marketCap']
price= data[list(symbol.split(",")).index(symbol)]['price']

print(marketCap/1000000000000)
#price
#tickerDf


2.980182335


In [85]:
df_columns = ['Ticker', 'Stock Price', 'Market Cap', '# of shares to buy']
sample_df = pd.DataFrame(columns = df_columns)
sample_df

,Ticker,Stock Price,Market Cap,# of shares to buy


In [86]:
sample_df.append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A'
        ],
        index=df_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Cap,# of shares to buy
0,AAPL,194.35,2980182335000,N/A


In [87]:
marketCap

2980182335000

In [88]:
# Loop thru every ticker in the csv
full_df = pd.DataFrame(columns=df_columns)
for symbol in stocks['Ticker']:
    symbol = symbol.replace(".", "-")
    data = yf.Ticker(symbol)
    tickerDf = data.history(period='1d', start='2024-05-30', end='2024-05-31')
    #print(symbol)
    marketCap= data.info['marketCap']
    price= round(tickerDf['Open'][0],2)
    full_df = full_df.append(
        pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A'
        ],
        index=df_columns
        ),
        ignore_index=True
    )
    
full_df

,Ticker,Stock Price,Market Cap,# of shares to buy
0,A,125.30,38176927744,N/A
1,AAL,11.76,7568000000,N/A
2,AAPL,190.76,2980182491136,N/A
3,ABBV,154.65,286318166016,N/A
4,ABNB,145.40,93372858368,N/A
...,...,...,...,...
498,XYL,138.05,33239486464,N/A
499,YUM,135.06,39862190080,N/A
500,ZBH,115.46,23337785344,N/A
501,ZBRA,319.55,15568250880,N/A


In [89]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [93]:
# 100 company chunks
symbol_chunks = list(chunks(stocks['Ticker'], 100))
#symbol_chunk
symbol_strings = []
for i in range(len(symbol_chunks)):
    symbol_strings.append(','.join(symbol_chunks[i]))
    
final_df = pd.DataFrame(columns=df_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_url = f'https://financialmodelingprep.com/api/v3/quote/{symbol_string}?apikey={token}'
    session = requests.session()
    request = session.get(api_url, timeout=15)
    data = request.json()
    
    for symbol in symbol_string.split(','):
        final_df = final_df.append(
            pd.Series(
            [
                symbol,
                data[list(symbol.split(",")).index(symbol)]['price'],
                data[list(symbol.split(",")).index(symbol)]['marketCap'],
                'N/A'
            ],
            index = df_columns),
            ignore_index=True
        )
        
final_df

,Ticker,Stock Price,Market Cap,# of shares to buy
0,A,194.35,2980182335000,N/A
1,AAL,194.35,2980182335000,N/A
2,AAPL,194.35,2980182335000,N/A
3,ABBV,194.35,2980182335000,N/A
4,ABNB,194.35,2980182335000,N/A
...,...,...,...,...
95,CINF,194.35,2980182335000,N/A
96,CL,194.35,2980182335000,N/A
97,CLX,194.35,2980182335000,N/A
98,CMA,194.35,2980182335000,N/A
